In [1]:

import scanpy as sc 
import pandas as pd 
import numpy as np 
import slideio
import matplotlib.pyplot as plt
import matplotlib
import cv2
from tqdm import tqdm 
import torch 
from shapely import Polygon, Point
from omicsio import datasets
import openslide
import slideio 
import os 
import warnings
import PIL 

warnings.simplefilter(action='ignore', category=Warning)


import geopandas as gpd

/dartfs/rc/nosnapshots/V/VaickusL-nb/EDIT_Students/users/Gokul_Srinivasan/anaconda3/envs/jupyter-ultimate/lib/python3.7/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
ST_image_map = {
    '/dartfs/rc/nosnapshots/V/VaickusL-nb/EDIT_Students/projects/visium_colon_9_27_23/visium_data/st_data/83-3':'/dartfs/rc/nosnapshots/V/VaickusL-nb/EDIT_Students/projects/visium_colon_9_27_23/wsi/_SS12251_074147.svs', 
    '/dartfs/rc/nosnapshots/V/VaickusL-nb/EDIT_Students/projects/visium_colon_9_27_23/visium_data/st_data/51_A10-38': '/dartfs/rc/nosnapshots/V/VaickusL-nb/EDIT_Students/projects/visium_colon_9_27_23/wsi/_SS12251_105411.svs', 
    '/dartfs/rc/nosnapshots/V/VaickusL-nb/EDIT_Students/projects/visium_colon_9_27_23/visium_data/st_data/116_A5-13_A7':'/dartfs/rc/nosnapshots/V/VaickusL-nb/EDIT_Students/projects/visium_colon_9_27_23/wsi/_SS12253_094205.svs', 
    '/dartfs/rc/nosnapshots/V/VaickusL-nb/EDIT_Students/projects/visium_colon_9_27_23/visium_data/st_data/80_A12-96_A8': '/dartfs/rc/nosnapshots/V/VaickusL-nb/EDIT_Students/projects/visium_colon_9_27_23/wsi/_SS12253_093632.svs', 
    '/dartfs/rc/nosnapshots/V/VaickusL-nb/EDIT_Students/projects/visium_colon_9_27_23/visium_data/st_data/127_A8-6': '/dartfs/rc/nosnapshots/V/VaickusL-nb/EDIT_Students/projects/visium_colon_9_27_23/wsi/_SS12251_105127.svs',
    '/dartfs/rc/nosnapshots/V/VaickusL-nb/EDIT_Students/projects/visium_colon_9_27_23/visium_data/st_data/97_A7-81_A11': '/dartfs/rc/nosnapshots/V/VaickusL-nb/EDIT_Students/projects/visium_colon_9_27_23/wsi/_SS12251_110624.svs',
    '/dartfs/rc/nosnapshots/V/VaickusL-nb/EDIT_Students/projects/visium_colon_9_27_23/visium_data/st_data/128_B9-102_B12': '/dartfs/rc/nosnapshots/V/VaickusL-nb/EDIT_Students/projects/visium_colon_9_27_23/wsi/_SS12253_093230.svs', 
    '/dartfs/rc/nosnapshots/V/VaickusL-nb/EDIT_Students/projects/visium_colon_9_27_23/visium_data/st_data/106_A4-60_B3': '/dartfs/rc/nosnapshots/V/VaickusL-nb/EDIT_Students/projects/visium_colon_9_27_23/wsi/_SS12251_110224.svs', 
    '/dartfs/rc/nosnapshots/V/VaickusL-nb/EDIT_Students/projects/visium_colon_9_27_23/visium_data/st_data/0_B11-10_A5': '/dartfs/rc/nosnapshots/V/VaickusL-nb/EDIT_Students/projects/visium_colon_9_27_23/wsi/_SS12253_093801.svs', 
    '/dartfs/rc/nosnapshots/V/VaickusL-nb/EDIT_Students/projects/visium_colon_9_27_23/visium_data/st_data/40_B10-18_G4': '/dartfs/rc/nosnapshots/V/VaickusL-nb/EDIT_Students/projects/visium_colon_9_27_23/wsi/_SS12251_110453.svs',
    '/dartfs/rc/nosnapshots/V/VaickusL-nb/EDIT_Students/projects/visium_colon_9_27_23/visium_data/st_data/11_C9-30_A6': '/dartfs/rc/nosnapshots/V/VaickusL-nb/EDIT_Students/projects/visium_colon_9_27_23/wsi/_SS12253_093109.svs', 
    '/dartfs/rc/nosnapshots/V/VaickusL-nb/EDIT_Students/projects/visium_colon_9_27_23/visium_data/st_data/91_A4-14_A4': '/dartfs/rc/nosnapshots/V/VaickusL-nb/EDIT_Students/projects/visium_colon_9_27_23/wsi/_SS12251_105941.svs', 
    '/dartfs/rc/nosnapshots/V/VaickusL-nb/EDIT_Students/projects/visium_colon_9_27_23/visium_data/st_data/100-52':'/dartfs/rc/nosnapshots/V/VaickusL-nb/EDIT_Students/projects/visium_colon_9_27_23/wsi/_SS12251_074004.svs',
    '/dartfs/rc/nosnapshots/V/VaickusL-nb/EDIT_Students/projects/visium_colon_9_27_23/visium_data/st_data/46_A31-111_B6': '/dartfs/rc/nosnapshots/V/VaickusL-nb/EDIT_Students/projects/visium_colon_9_27_23/wsi/_SS12251_091934.svs', 
    '/dartfs/rc/nosnapshots/V/VaickusL-nb/EDIT_Students/projects/visium_colon_9_27_23/visium_data/st_data/36_B19-49_A13': '/dartfs/rc/nosnapshots/V/VaickusL-nb/EDIT_Students/projects/visium_colon_9_27_23/wsi/_SS12251_092229.svs', 
    '/dartfs/rc/nosnapshots/V/VaickusL-nb/EDIT_Students/projects/visium_colon_9_27_23/visium_data/st_data/28_A9-74_A3': '/dartfs/rc/nosnapshots/V/VaickusL-nb/EDIT_Students/projects/visium_colon_9_27_23/wsi/_SS12251_105645.svs', 
    '/dartfs/rc/nosnapshots/V/VaickusL-nb/EDIT_Students/projects/all_visium_data/colon/CytAssist_x8/data/analysis/092534': '/dartfs/rc/nosnapshots/V/VaickusL-nb/EDIT_Students/projects/all_visium_data/colon/CytAssist_x8/images/_SS12251_092534.svs', 
    '/dartfs/rc/nosnapshots/V/VaickusL-nb/EDIT_Students/projects/all_visium_data/colon/CytAssist_x8/data/analysis/091759': '/dartfs/rc/nosnapshots/V/VaickusL-nb/EDIT_Students/projects/all_visium_data/colon/CytAssist_x8/images/_SS12251_091759.svs', 
    '/dartfs/rc/nosnapshots/V/VaickusL-nb/EDIT_Students/projects/all_visium_data/colon/CytAssist_x8/data/analysis/092842': '/dartfs/rc/nosnapshots/V/VaickusL-nb/EDIT_Students/projects/all_visium_data/colon/CytAssist_x8/images/_SS12251_092842.svs', 
    '/dartfs/rc/nosnapshots/V/VaickusL-nb/EDIT_Students/projects/all_visium_data/colon/CytAssist_x8/data/analysis/092146': '/dartfs/rc/nosnapshots/V/VaickusL-nb/EDIT_Students/projects/all_visium_data/colon/CytAssist_x8/images/_SS12251_092146.svs', 
    '/dartfs/rc/nosnapshots/V/VaickusL-nb/EDIT_Students/projects/all_visium_data/colon/autostainer/data': '/dartfs/rc/nosnapshots/V/VaickusL-nb/EDIT_Students/projects/all_visium_data/colon/autostainer/images/_SS12254_081342_40x.tiff', 

}



ST_tissue_map = {
    '/dartfs/rc/nosnapshots/V/VaickusL-nb/EDIT_Students/projects/visium_colon_9_27_23/visium_data/st_data/83-3':'/dartfs/rc/nosnapshots/V/VaickusL-nb/EDIT_Students/users/Gokul_Srinivasan/Colon_ST_Training/colon_tissue_annotations/_SS12251_074147.geojson', 
    '/dartfs/rc/nosnapshots/V/VaickusL-nb/EDIT_Students/projects/visium_colon_9_27_23/visium_data/st_data/51_A10-38': '/dartfs/rc/nosnapshots/V/VaickusL-nb/EDIT_Students/users/Gokul_Srinivasan/Colon_ST_Training/colon_tissue_annotations/_SS12251_105411.geojson', 
    '/dartfs/rc/nosnapshots/V/VaickusL-nb/EDIT_Students/projects/visium_colon_9_27_23/visium_data/st_data/116_A5-13_A7':'/dartfs/rc/nosnapshots/V/VaickusL-nb/EDIT_Students/users/Gokul_Srinivasan/Colon_ST_Training/colon_tissue_annotations/_SS12253_094205.geojson', 
    '/dartfs/rc/nosnapshots/V/VaickusL-nb/EDIT_Students/projects/visium_colon_9_27_23/visium_data/st_data/80_A12-96_A8': '/dartfs/rc/nosnapshots/V/VaickusL-nb/EDIT_Students/users/Gokul_Srinivasan/Colon_ST_Training/colon_tissue_annotations/_SS12253_093632.geojson', 
    '/dartfs/rc/nosnapshots/V/VaickusL-nb/EDIT_Students/projects/visium_colon_9_27_23/visium_data/st_data/127_A8-6': '/dartfs/rc/nosnapshots/V/VaickusL-nb/EDIT_Students/users/Gokul_Srinivasan/Colon_ST_Training/colon_tissue_annotations/_SS12251_105127.geojson',
    '/dartfs/rc/nosnapshots/V/VaickusL-nb/EDIT_Students/projects/visium_colon_9_27_23/visium_data/st_data/97_A7-81_A11': '/dartfs/rc/nosnapshots/V/VaickusL-nb/EDIT_Students/users/Gokul_Srinivasan/Colon_ST_Training/colon_tissue_annotations/_SS12251_110624.geojson',
    '/dartfs/rc/nosnapshots/V/VaickusL-nb/EDIT_Students/projects/visium_colon_9_27_23/visium_data/st_data/128_B9-102_B12': '/dartfs/rc/nosnapshots/V/VaickusL-nb/EDIT_Students/users/Gokul_Srinivasan/Colon_ST_Training/colon_tissue_annotations/_SS12253_093230.geojson', 
    '/dartfs/rc/nosnapshots/V/VaickusL-nb/EDIT_Students/projects/visium_colon_9_27_23/visium_data/st_data/106_A4-60_B3': '/dartfs/rc/nosnapshots/V/VaickusL-nb/EDIT_Students/users/Gokul_Srinivasan/Colon_ST_Training/colon_tissue_annotations/_SS12251_110224.geojson', 
    '/dartfs/rc/nosnapshots/V/VaickusL-nb/EDIT_Students/projects/visium_colon_9_27_23/visium_data/st_data/0_B11-10_A5': '/dartfs/rc/nosnapshots/V/VaickusL-nb/EDIT_Students/users/Gokul_Srinivasan/Colon_ST_Training/colon_tissue_annotations/_SS12253_093801.geojson', 
    '/dartfs/rc/nosnapshots/V/VaickusL-nb/EDIT_Students/projects/visium_colon_9_27_23/visium_data/st_data/40_B10-18_G4': '/dartfs/rc/nosnapshots/V/VaickusL-nb/EDIT_Students/users/Gokul_Srinivasan/Colon_ST_Training/colon_tissue_annotations/_SS12251_110453.geojson',
    '/dartfs/rc/nosnapshots/V/VaickusL-nb/EDIT_Students/projects/visium_colon_9_27_23/visium_data/st_data/11_C9-30_A6': '/dartfs/rc/nosnapshots/V/VaickusL-nb/EDIT_Students/users/Gokul_Srinivasan/Colon_ST_Training/colon_tissue_annotations/_SS12253_093109.geojson', 
    '/dartfs/rc/nosnapshots/V/VaickusL-nb/EDIT_Students/projects/visium_colon_9_27_23/visium_data/st_data/91_A4-14_A4': '/dartfs/rc/nosnapshots/V/VaickusL-nb/EDIT_Students/users/Gokul_Srinivasan/Colon_ST_Training/colon_tissue_annotations/_SS12251_105941.geojson', 
    '/dartfs/rc/nosnapshots/V/VaickusL-nb/EDIT_Students/projects/visium_colon_9_27_23/visium_data/st_data/100-52':'/dartfs/rc/nosnapshots/V/VaickusL-nb/EDIT_Students/users/Gokul_Srinivasan/Colon_ST_Training/colon_tissue_annotations/_SS12251_074004.geojson',
    '/dartfs/rc/nosnapshots/V/VaickusL-nb/EDIT_Students/projects/visium_colon_9_27_23/visium_data/st_data/46_A31-111_B6': '/dartfs/rc/nosnapshots/V/VaickusL-nb/EDIT_Students/users/Gokul_Srinivasan/Colon_ST_Training/colon_tissue_annotations/_SS12251_091934.geojson', 
    '/dartfs/rc/nosnapshots/V/VaickusL-nb/EDIT_Students/projects/visium_colon_9_27_23/visium_data/st_data/36_B19-49_A13': '/dartfs/rc/nosnapshots/V/VaickusL-nb/EDIT_Students/users/Gokul_Srinivasan/Colon_ST_Training/colon_tissue_annotations/_SS12251_092229.geojson', 
    '/dartfs/rc/nosnapshots/V/VaickusL-nb/EDIT_Students/projects/visium_colon_9_27_23/visium_data/st_data/28_A9-74_A3': '/dartfs/rc/nosnapshots/V/VaickusL-nb/EDIT_Students/users/Gokul_Srinivasan/Colon_ST_Training/colon_tissue_annotations/_SS12251_105645.geojson', 
    '/dartfs/rc/nosnapshots/V/VaickusL-nb/EDIT_Students/projects/all_visium_data/colon/CytAssist_x8/data/analysis/092534': '/dartfs/rc/nosnapshots/V/VaickusL-nb/EDIT_Students/users/Gokul_Srinivasan/Colon_ST_Training/colon_tissue_annotations/_SS12251_092534.geojson', 
    '/dartfs/rc/nosnapshots/V/VaickusL-nb/EDIT_Students/projects/all_visium_data/colon/CytAssist_x8/data/analysis/091759': '/dartfs/rc/nosnapshots/V/VaickusL-nb/EDIT_Students/users/Gokul_Srinivasan/Colon_ST_Training/colon_tissue_annotations/_SS12251_091759.geojson', 
    '/dartfs/rc/nosnapshots/V/VaickusL-nb/EDIT_Students/projects/all_visium_data/colon/CytAssist_x8/data/analysis/092842': '/dartfs/rc/nosnapshots/V/VaickusL-nb/EDIT_Students/users/Gokul_Srinivasan/Colon_ST_Training/colon_tissue_annotations/_SS12251_092842.geojson', 
    '/dartfs/rc/nosnapshots/V/VaickusL-nb/EDIT_Students/projects/all_visium_data/colon/CytAssist_x8/data/analysis/092146': '/dartfs/rc/nosnapshots/V/VaickusL-nb/EDIT_Students/users/Gokul_Srinivasan/Colon_ST_Training/colon_tissue_annotations/_SS12251_092146.geojson', 
    '/dartfs/rc/nosnapshots/V/VaickusL-nb/EDIT_Students/projects/all_visium_data/colon/autostainer/data': "None", 
}


In [3]:
def produce_slide_data(adata_path, adatas_dic, img_save_path, matrices_save_path, metadata_save_path, adata_save_path): 
    sample_id = adata_path.split("/")[-1] #get sample_id 
    
    annotation_path = ST_tissue_map[adata_path]
    image_path = ST_image_map[adata_path]
    
    full_image = None
    adatas = None
    
    #check if this is the autostainer slide not and load the image accordingly
    if adata_path != '/dartfs/rc/nosnapshots/V/VaickusL-nb/EDIT_Students/projects/all_visium_data/colon/autostainer/data': 
        full_image = load_image(image_path)
    else: #this is the autostainer slide
        PIL.Image.MAX_IMAGE_PIXELS = None 
        full_image = np.array(PIL.Image.open(image_path))
        
    #likewise, check if this is the autostainer slide and don't trim the adata 
    if adata_path != '/dartfs/rc/nosnapshots/V/VaickusL-nb/EDIT_Students/projects/all_visium_data/colon/autostainer/data': 
        adatas = custom_load_visium(adata_path, adatas_dic)
    else: #this is the autostainer slide
        adatas = [adatas_dic["data_0_adata"]]        

    slice_id = 0 
    
    for adata in adatas: 
        #save images and make metadata
        filepaths = []
        patch_indices = []
        sizes = []
        xs = []
        ys = []
        spot_ids = []
        metadata = pd.DataFrame()

        i = 0 
        for (x,y), spot_id in tqdm(zip(adata.obsm["spatial"], list(adata.obs.index))): 
            x *= 2
            y *= 2
            np_patch = make_patch(x, y, full_image, size = 512)
            path = str(img_save_path + sample_id + "_" + str(slice_id) + "_" + str(i) + ".npy")
            np.save(path, np_patch)
            filepaths.append(path)
            patch_indices.append(i)
            spot_ids.append(spot_id)
            sizes.append(512)
            xs.append(x)
            ys.append(y)
            i += 1

        metadata["size"] = sizes
        metadata["index"] = patch_indices
        metadata["filepath"] = filepaths
        metadata["x"] = xs
        metadata["y"] = ys
        metadata["spot_id"] = spot_ids

        #save metadata
        metadata.to_csv(str(metadata_save_path + sample_id +  "_" + str(slice_id) + "_metadata.csv"))
        
        #save gxp
        # we don't need to norm or log b/c we already did this elsewhere
        gxp_matrix = adata.X.toarray() # we will transform this into log expression matrix
        np.save(str(matrices_save_path + sample_id + "_" + str(slice_id) + "_gxp_matrix.npy"), gxp_matrix)
        
        #save adata 
        adata.write(str(adata_save_path + sample_id + "_" + str(slice_id) + "_adata.h5ad"))
        slice_id += 1 #this tracks which of the tissue slices


In [4]:

def custom_load_visium(path, adatas_dic): 
    sid = path.split("/")[-1]
    adatas = []
    for i in range(2): 
        adatas.append(adatas_dic[f"{sid}_{i}_adata"])
    return adatas


def trim_and_split(path, adata): 
    annots = gpd.read_file(path)
    tissue_masks = [a for a in annots["geometry"]]
    #split the adatas into two seperate and trim 
    adatas = []
    for t in tqdm(tissue_masks): #for each tissue mask, generate a seperate adata object where the spots exist just inside the tissue mask
        t = t.buffer(50)
        i = 0 
        bool_arr = np.zeros(len(adata.obsm["spatial"]))
        for x,y in adata.obsm["spatial"]:
            p = Point(x*2, y*2)
            if t.contains(p): 
                bool_arr[i] = 1
            i += 1
        bool_arr = bool_arr.astype(bool)
        subset = adata[bool_arr]
        adatas.append(subset)
    
    return adatas 

In [5]:
def load_image(img_path: str):
    slide = slideio.open_slide(img_path,'SVS')
    num_scenes = slide.num_scenes
    scene = slide.get_scene(0)
    full_img = scene.read_block()
    np_im =  np.asarray(full_img) 
    return np_im


def make_patch(x, y, np_im, size): #np is indexed y,x 
    np_patch = np_im[int(y-size/2):int(y+size/2), int(x-size/2): int(x+size/2)]
    return np_patch


In [6]:
def produce_all() -> None: 
    folder_name = "all_genes_filtered"
    
    img_save_path = f"/dartfs/rc/nosnapshots/V/VaickusL-nb/EDIT_Students/users/Gokul_Srinivasan/Colon_ST_Training/gene_selection/{folder_name}/data/images/"
    matrices_save_path = f"/dartfs/rc/nosnapshots/V/VaickusL-nb/EDIT_Students/users/Gokul_Srinivasan/Colon_ST_Training/gene_selection/{folder_name}/data/matrices/"
    metadata_save_path = f"/dartfs/rc/nosnapshots/V/VaickusL-nb/EDIT_Students/users/Gokul_Srinivasan/Colon_ST_Training/gene_selection/{folder_name}/data/metadata/"
    adata_save_path = f"/dartfs/rc/nosnapshots/V/VaickusL-nb/EDIT_Students/users/Gokul_Srinivasan/Colon_ST_Training/gene_selection/{folder_name}/data/adatas/"
    
    adatas_dic = process_all_adatas()

    for adata_path in tqdm(ST_image_map): #iterate through all visium 
        sample_id = adata_path.split("/")[-1] 
        if os.path.exists(str(adata_save_path + sample_id + "_" + str(0) + "_adata.h5ad")) or os.path.exists(str(adata_save_path + sample_id + "_" + str(1) + "_adata.h5ad")):
            print(f"skipping: {sample_id}")
            continue
        else: 
            produce_slide_data(adata_path, adatas_dic, img_save_path, matrices_save_path, metadata_save_path, adata_save_path)

In [7]:
def preprocess_visium_data(adata, min_genes_per_spot, min_spots_per_gene, remove_mitochondrial=True):
    adata = adata.copy()
    # Filter spots with fewer than min_genes_per_spot
    sc.pp.filter_cells(adata, min_genes=min_genes_per_spot)

    # Filter genes detected in fewer than min_spots_per_gene
    sc.pp.filter_genes(adata, min_cells=min_spots_per_gene)

    # Optionally, remove mitochondrial genes
    if remove_mitochondrial:
        # Assuming mitochondrial genes are prefixed with "MT-" (this can vary)
        mt_genes = [gene for gene in adata.var_names if gene.startswith('MT-')]
        adata = adata[:, ~adata.var_names.isin(mt_genes)].copy()
    
    return adata

In [8]:
def process_all_adatas(): 
    base_path = "/dartfs/rc/nosnapshots/V/VaickusL-nb/EDIT_Students/users/Gokul_Srinivasan/Colon_ST_Training/gene_selection/all_genes/data/adatas"
    
    adatas = {}
    for f in os.listdir(base_path):
        adatas[f.split(".")[0]] = sc.read_h5ad(base_path + "/" + f)
        adatas[f.split(".")[0]].var_names_make_unique()
    

    for k in adatas:
        adatas[k].obs["sample_id"] = k
        adatas[k].obs["sample_id"] = adatas[k].obs["sample_id"].astype("category")
    
    # combine all the adatas and filter simeltaneously 
    adatas_arr = [adatas[k] for k in adatas]
    adata_combined = adatas_arr[0].concatenate(adatas_arr[1:])
    sc.pp.calculate_qc_metrics(adata_combined, inplace=True)
    ac_filtered = preprocess_visium_data(adata_combined, min_genes_per_spot = np.percentile(adata_combined.obs['total_counts'], 5), min_spots_per_gene = np.percentile(adata_combined.var['total_counts'], 5), remove_mitochondrial=True)
    
    print(len(adata_combined), len(ac_filtered))

    # Splitting the combined AnnData object back into individual datasets
    individual_adatas = {}
    unique_samples = ac_filtered.obs['sample_id'].unique()

    for sample_id in unique_samples:
        individual_adatas[sample_id] = ac_filtered[ac_filtered.obs['sample_id'] == sample_id].copy()
    
    return individual_adatas

In [ ]:
# produce_all()

275658 229113


  0%|                                                                      | 0/21 [00:00<?, ?it/s]
0it [00:00, ?it/s]
4it [00:00, 32.99it/s]
8it [00:00, 33.75it/s]
12it [00:00, 33.90it/s]
16it [00:00, 34.16it/s]
20it [00:00, 32.79it/s]
24it [00:00, 32.10it/s]
28it [00:00, 30.77it/s]
32it [00:00, 31.65it/s]
36it [00:01, 32.29it/s]
40it [00:01, 32.88it/s]
44it [00:01, 32.62it/s]
48it [00:01, 32.57it/s]
52it [00:01, 32.47it/s]
56it [00:01, 30.25it/s]
60it [00:01, 31.32it/s]
64it [00:01, 32.26it/s]
68it [00:02, 32.85it/s]
72it [00:02, 30.33it/s]
76it [00:02, 27.46it/s]
79it [00:02, 27.30it/s]
82it [00:02, 27.73it/s]
86it [00:02, 29.09it/s]
89it [00:02, 29.17it/s]
92it [00:02, 28.98it/s]
96it [00:03, 30.13it/s]
100it [00:03, 31.33it/s]
104it [00:03, 26.68it/s]
107it [00:03, 25.18it/s]
111it [00:03, 26.90it/s]
115it [00:03, 28.79it/s]
119it [00:03, 30.28it/s]
123it [00:04, 31.38it/s]
127it [00:04, 32.27it/s]
131it [00:04, 33.05it/s]
135it [00:04, 33.34it/s]
139it [00:04, 32.33it/s]
143it [00

In [9]:
# # for paper 

# base_path = "/dartfs/rc/nosnapshots/V/VaickusL-nb/EDIT_Students/users/Gokul_Srinivasan/Colon_ST_Training/gene_selection/all_genes/data/adatas"
    
# adatas = {}
# for f in os.listdir(base_path):
#     adatas[f.split(".")[0]] = sc.read_h5ad(base_path + "/" + f)
#     adatas[f.split(".")[0]].var_names_make_unique()


# for k in adatas:
#     adatas[k].obs["sample_id"] = k
#     adatas[k].obs["sample_id"] = adatas[k].obs["sample_id"].astype("category")

# # combine all the adatas and filter simeltaneously 
# adatas_arr = [adatas[k] for k in adatas]
# adata_combined = adatas_arr[0].concatenate(adatas_arr[1:])
# sc.pp.calculate_qc_metrics(adata_combined, inplace=True)
# print(np.percentile(adata_combined.obs['total_counts'], 5), np.percentile(adata_combined.var['total_counts'], 5))
# ac_filtered = preprocess_visium_data(adata_combined, min_genes_per_spot = np.percentile(adata_combined.obs['total_counts'], 5), min_spots_per_gene = np.percentile(adata_combined.var['total_counts'], 5), remove_mitochondrial=True)

1963.47578125 3584.462255859375


In [10]:
# adata_combined.shape

(275658, 18085)

In [11]:
# ac_filtered.shape

(229113, 17796)

In [12]:
# # Splitting the combined AnnData object back into individual datasets
# individual_adatas = {}
# unique_samples = ac_filtered.obs['sample_id'].unique()

# for sample_id in unique_samples:
#     individual_adatas[sample_id] = ac_filtered[ac_filtered.obs['sample_id'] == sample_id].copy()


In [19]:
# exclude_list = ["51_A10-38_0", "127_A8-6_0", "36_B19-49_A13_0"]
# new_adatas_arr = [individual_adatas[k] for k in individual_adatas if "_".join(k.split("_")[0:-1]) not in exclude_list]

In [20]:
# new_adatas_arr = new_adatas_arr[0].concatenate(new_adatas_arr[1:])

In [21]:
# new_adatas_arr.shape

(213036, 17796)